# GEOtiled Accuracy Test Notebook

This notebook computes the mean squared error (MSE) and absolute different between data generated by GDAL/SAGA vs data generated using GEOtiled.

## Preconfiguration

Import the needed libraries to run the code blocks.

In [1]:
import compute_functions as cm
import accuracy_functions as ac
import geotiled
import time
import os

Modify the testing parameters below to change the data parameters are computed from and options for GEOtiled.

In [2]:
# The working directory to store files in
working_directory = '/media/volume/geotiled-saga/accuracy_test'
geotiled.set_working_directory(working_directory)

# Data to download
resolution = '30m'
region = 'TN'

# Size of tiles to produce specified as [width,height] (units of pixels)
tile_size = [4569,4216]

# Parameter to compute and method to use
parameter = 'slope'
method = 'SAGA'

## Initialization

Download and preprocess the data.

In [4]:
# Download the desired data
geotiled.fetch_dem(shapefile=region, dataset=resolution, save_to_txt=False, download=True)

# Mosaic DEMs
geotiled.build_mosaic(input_folder='dem_tiles', output_file='mosaic.tif', description='Elevation')

# Reproject mosaic into computable format
geotiled.reproject(input_file='mosaic.tif', output_file='elevation.tif', projection='EPSG:26918')



ownloading: 100%|██████████████████████████████████████████████| 1.72G/1.72G [00:16<00:00, 104MB/s]

In [3]:
# Set paths to input and output file
input_file = os.path.join(os.getcwd(), 'elevation.tif')
seq_output_file = os.path.join(os.getcwd(), parameter+'.tif')
geotiled_output_file = os.path.join(os.getcwd(), 'geotiled_'+parameter+'.tif')

## Compute Sequentially

Compute the parameter with the sequential method.

In [4]:
start_time = time.time()
cm.compute_parameter(input_file, seq_output_file, parameter, method)
end_time = time.time() - start_time
print('Compute Time(s):', end_time)

Compute Time(s): 129.01918530464172


## Compute with GEOtiled

Compute the parameter using GEOtiled.

In [5]:
start_time = time.time()
geotiled.crop_and_compute(input_file=input_file, column_length=tile_size[0], row_length=tile_size[1], parameter_list=[parameter], compute_method=method)
geotiled.build_mosaic(input_folder='unbuffered_slope_tiles', output_file=geotiled_output_file)
end_time = time.time() - start_time
print('Compute Time(s):', end_time)

Compute Time(s): 29.759188652038574


## Compare Final Data

Compute MSE and absolute difference between generated data.

In [6]:
ad = ac.absolute_difference(seq_output_file, geotiled_output_file)
mse = ac.mse(seq_output_file, geotiled_output_file)
print('Absolute Difference:', ad)
print('MSE:', mse)

Absolute Difference: 0.0
MSE: 0.0


### End of Notebook